<a href="https://colab.research.google.com/github/A5HU705H/odin-landing-page/blob/main/techfest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# backtesting.py for providing backtesting interface and ta-lib for builtin technical indicator supports
! pip3 install backtesting
! pip3 install ta-lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=5e5cdd73e5b49a216c61a8f20b0f69d271ce3a0a60231ae589cb0b6a90d292bc
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ta-lib
Failed to build ta-lib
ERRO

In [2]:
# talib import fix
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib

--2023-12-11 17:52:44--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-12-11 17:52:44--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-12-11 17:52:44--  http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving cfhcable.dl.s

In [3]:
import numpy as np
import pandas as pd

In [103]:
# we are given data.csv, we add the headings for required data
data = pd.read_csv('/content/data.csv')
headers = ['date', 'expiry_date', 'timestamp', 'Open', 'High', 'Low', 'Close']
data.columns = headers

In [104]:
data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%H%M')
data['date'] = data['date'].astype(str) + data['timestamp'].astype(str).str[10:]
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
data['expiry_date'] = pd.to_datetime(data['expiry_date'], format='%Y%m%d')
data = data.set_index('date')  # Set the index
data.drop('timestamp', axis = 1)

,expiry_date,Open,High,Low,Close
date,,,,,
2020-01-01 09:17:00,2020-01-30,30809.99476,30822.24503,30801.45383,30819.20530
2020-01-01 09:18:00,2020-01-30,30819.20530,30822.05503,30814.74177,30821.39004
2020-01-01 09:19:00,2020-01-30,30821.39004,30831.22525,30803.39885,30804.67985
2020-01-01 09:20:00,2020-01-30,30806.43546,30810.75419,30793.86586,30802.49747
2020-01-01 09:21:00,2020-01-30,30802.49747,30811.79845,30799.03476,30802.40259
...,...,...,...,...,...
2022-12-30 15:26:00,2023-01-25,23110.54002,23144.45146,23108.56403,23135.66988
2022-12-30 15:27:00,2023-01-25,23133.20795,23134.09097,23110.70025,23111.47473
2022-12-30 15:28:00,2023-01-25,23111.82193,23121.38728,23104.82660,23121.38728


In [42]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(talib.SMA, close, self.n1)
        self.sma2 = self.I(talib.SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()

In [105]:
bt = Backtest(data, SmaCross, cash = 100_000)
stats = bt.run()

In [106]:
stats

Start                     2020-01-01 09:17:00
End                       2022-12-30 15:30:00
Duration                   1094 days 06:13:00
Exposure Time [%]                   52.195959
Equity Final [$]                 104242.65193
Equity Peak [$]                  153088.24846
Return [%]                           4.242652
Buy & Hold Return [%]              -24.947233
Return (Ann.) [%]                    1.330666
Volatility (Ann.) [%]               18.232544
Sharpe Ratio                         0.072983
Sortino Ratio                        0.125017
Calmar Ratio                         0.039848
Max. Drawdown [%]                  -33.393432
Avg. Drawdown [%]                   -0.624227
Max. Drawdown Duration      952 days 02:04:00
Avg. Drawdown Duration        5 days 11:18:00
# Trades                                 8055
Win Rate [%]                        36.188703
Best Trade [%]                       7.463483
Worst Trade [%]                     -3.143408
Avg. Trade [%]                    

In [99]:
bt.plot()

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passin

GridPlot(id='p2508', ...)

In [47]:
import yfinance as yf
from backtesting.test import SMA, GOOG
bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1868', ...)

In [50]:
GOOG


,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800
